In [1]:
import os
import cv2
import numpy as np
import dlib
import joblib
import mediapipe as mp
import time
from tkinter import Tk
from tkinter.filedialog import askopenfilename

# Initialize dlib's face detector and shape predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Function to detect facial landmarks from an image
def get_landmarks(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    rects = detector(gray, 1)
    
    if len(rects) > 0:
        # Assuming one face per image
        shape = predictor(gray, rects[0])
        landmarks = np.array([[p.x, p.y] for p in shape.parts()])
        return landmarks.flatten()  # Flatten the landmarks into a 1D array
    else:
        return None

# Load your pre-trained SVM models
svr_pitch = joblib.load('svr_pitch_model.pkl')
svr_yaw = joblib.load('svr_yaw_model.pkl')
svr_roll = joblib.load('svr_roll_model.pkl')

# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(max_num_faces=1, min_detection_confidence=0.5)

mp_drawing = mp.solutions.drawing_utils
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

# Set up tkinter root (hidden)
Tk().withdraw()

# Open file dialog to select an image
image_path = askopenfilename(title="Select an Image", filetypes=[("Image Files", "*.jpg;*.jpeg;*.png")])
if not image_path:
    print("No file selected.")
    exit()

# Read and process the selected image
image = cv2.imread(image_path)
if image is None:
    print("Error reading image.")
    exit()

# Get landmarks using dlib and predict angles using SVM models
landmarks = get_landmarks(image)
if landmarks is not None:
    pitch_pred = svr_pitch.predict([landmarks])[0]
    yaw_pred = svr_yaw.predict([landmarks])[0]
    roll_pred = svr_roll.predict([landmarks])[0]
    print(f'Predicted by SVM - Pitch: {pitch_pred:.2f}, Yaw: {yaw_pred:.2f}, Roll: {roll_pred:.2f}')
else:
    print("No face detected using dlib.")
    exit()

# Use MediaPipe to get face landmarks and calculate pose direction
start = time.time()

# Convert BGR to RGB
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image_rgb.flags.writeable = False
results = face_mesh.process(image_rgb)
image_rgb.flags.writeable = True
image = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)

img_h, img_w, img_c = image.shape

if results.multi_face_landmarks:
    for face_idx, face_landmarks in enumerate(results.multi_face_landmarks):
        face_3d = []
        face_2d = []
        
        for idx, lm in enumerate(face_landmarks.landmark):
            if idx == 33 or idx == 263 or idx == 1 or idx == 61 or idx == 291 or idx == 199:
                if idx == 1:
                    nose_2d = (lm.x * img_w, lm.y * img_h)
                    
                x, y = int(lm.x * img_w), int(lm.y * img_h)
                
                # 2D coordinates
                face_2d.append([x, y])
                
                # 3D coordinates
                face_3d.append([x, y, lm.z])
        
        # Convert lists to numpy arrays
        face_2d = np.array(face_2d, dtype=np.float64)
        face_3d = np.array(face_3d, dtype=np.float64)
        
        # Rotation degree
        x = pitch_pred * 180 / np.pi  # Convert pitch to degrees
        y = yaw_pred * 180 / np.pi    # Invert yaw to correct the left-right direction
        z = roll_pred * 180 / np.pi   # Convert roll to degrees
        
        # Tilting position
        # if y < -10:
        #     text = f"Face {face_idx + 1}: Looking Left"
        # elif y > 10:
        #     text = f"Face {face_idx + 1}: Looking Right"
        # elif x < -10:
        #     text = f"Face {face_idx + 1}: Looking Down"
        # elif x > 10:
        #     text = f"Face {face_idx + 1}: Looking Up"
        # else:
        #     text = f"Face {face_idx + 1}: Forward"
        
        # Nose direction
        length = 100
        p1 = (int(nose_2d[0]), int(nose_2d[1]))
        p2 = (int(nose_2d[0] + length * np.sin(y * np.pi / 180)), 
              int(nose_2d[1] - length * np.sin(x * np.pi / 180))) 
        
        # Draw axis lines
        cv2.line(image, p1, p2, (255, 0, 0), 3)  # Blue line for nose direction
        cv2.line(image, p1, (p1[0], p1[1] + 100), (0, 255, 0), 3)  # Green line
        cv2.line(image, p1, (p1[0] + 100, p1[1]), (0, 0, 255), 3)  # Red line
        
        # # Text on image with smaller font size
        font_scale = 0.4
        thickness = 1
        
        # cv2.putText(image, text, (50, 50 + face_idx * 110), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (0, 0, 255), thickness)
        # cv2.putText(image, f"X: {x:.2f} Y: {y:.2f} Z: {z:.2f}", (50, 80 + face_idx * 110), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (0, 0, 255), thickness)
        cv2.putText(image, f"SVM Pitch: {pitch_pred:.2f}, Yaw: {yaw_pred:.2f}, Roll: {roll_pred:.2f}", (50, 110 + face_idx * 110), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (0, 0, 255), thickness)
    
    end = time.time()
    totalTime = end - start
    
    fps = 1 / totalTime
    print("FPS: ", fps)
    
    cv2.putText(image, f"FPS: {int(fps)}", (50, 140 + len(results.multi_face_landmarks) * 110), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (0, 0, 255), thickness)
    cv2.imshow("Head Pose Estimation", image)
    cv2.waitKey(0)  # Wait for a key press to close the image window

cv2.destroyAllWindows()

Predicted by SVM - Pitch: -0.23, Yaw: -0.58, Roll: -0.11
FPS:  99.93814482117753


c:\Users\0king\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [7]:
import os
import cv2
import numpy as np
import dlib
import joblib
import mediapipe as mp
from tkinter import Tk
from tkinter.filedialog import askopenfilename

# Initialize dlib's face detector and shape predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Function to detect facial landmarks from an image
def get_landmarks(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    rects = detector(gray, 1)
    
    landmarks_list = []
    if len(rects) > 0:
        for rect in rects[:5]:  # Limit to the first 5 faces
            shape = predictor(gray, rect)
            landmarks = np.array([[p.x, p.y] for p in shape.parts()])
            landmarks_list.append(landmarks.flatten())  # Flatten the landmarks into a 1D array
        return landmarks_list
    else:
        return []

# Load your pre-trained SVM models
svr_pitch = joblib.load('svr_pitch_model.pkl')
svr_yaw = joblib.load('svr_yaw_model.pkl')
svr_roll = joblib.load('svr_roll_model.pkl')

# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(max_num_faces=5, min_detection_confidence=0.2)

mp_drawing = mp.solutions.drawing_utils
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

# Set up tkinter root (hidden)
Tk().withdraw()

# Open file dialog to select a video
video_path = askopenfilename(title="Select a Video File", filetypes=[("Video Files", "*.mp4;*.avi;*.mov")])
if not video_path:
    print("No file selected.")
    exit()

# Read and process the selected video
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Error opening video.")
    exit()

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Output video writer
output_video_path = os.path.splitext(video_path)[0] + "_output.avi"
out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc('M','J','P','G'), fps, (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Get landmarks using dlib and predict angles using SVM models
    landmarks_list = get_landmarks(frame)
    if not landmarks_list:
        continue

    for landmarks in landmarks_list:
        pitch_pred = svr_pitch.predict([landmarks])[0]
        yaw_pred = svr_yaw.predict([landmarks])[0]
        roll_pred = svr_roll.predict([landmarks])[0]

        # Use MediaPipe to get face landmarks and calculate pose direction
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(image_rgb)
        image_rgb.flags.writeable = True
        img_h, img_w, img_c = frame.shape

        if results.multi_face_landmarks:
            for face_idx, face_landmarks in enumerate(results.multi_face_landmarks):
                face_3d = []
                face_2d = []
                
                for idx, lm in enumerate(face_landmarks.landmark):
                    if idx == 33 or idx == 263 or idx == 1 or idx == 61 or idx == 291 or idx == 199:
                        if idx == 1:
                            nose_2d = (lm.x * img_w, lm.y * img_h)
                            
                        x, y = int(lm.x * img_w), int(lm.y * img_h)
                        
                        # 2D coordinates
                        face_2d.append([x, y])
                        
                        # 3D coordinates
                        face_3d.append([x, y, lm.z])
                
                # Convert lists to numpy arrays
                face_2d = np.array(face_2d, dtype=np.float64)
                face_3d = np.array(face_3d, dtype=np.float64)
                
                # Rotation degree (predicted by SVM)
                x = pitch_pred * 180 / np.pi  # Convert pitch to degrees
                y = -yaw_pred * 180 / np.pi    # Convert yaw to degrees
                z = roll_pred * 180 / np.pi   # Convert roll to degrees
                
                # Nose direction
                length = 100
                p1 = (int(nose_2d[0]), int(nose_2d[1]))
                p2 = (int(nose_2d[0] + length * np.sin(y * np.pi / 180)), 
                      int(nose_2d[1] - length * np.sin(x * np.pi / 180))) 
                
                # Draw axis lines
                cv2.line(frame, p1, p2, (255, 0, 0), 3)  # Blue line for nose direction
                cv2.line(frame, p1, (p1[0], p1[1] + 100), (0, 255, 0), 3)  # Green line
                cv2.line(frame, p1, (p1[0] + 100, p1[1]), (0, 0, 255), 3)  # Red line
                
                # Text on image with smaller font size
                font_scale = 0.4
                thickness = 1
                #cv2.putText(frame, f"SVM Pitch: {pitch_pred:.2f}, Yaw: {yaw_pred:.2f}, Roll: {roll_pred:.2f}", 
                            #(50, 50 + face_idx * 60), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (0, 0, 255), thickness)

    # Write the processed frame to the output video
    out.write(frame)

# Release everything
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Video saved as {output_video_path}")

c:\Users\0king\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\0king\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\0king\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabas

Video saved as C:/Users/0king/Desktop/WhatsApp Video 2024-08-22 at 20.44.42_370713d1_output.avi


In [3]:
import os
import cv2
import numpy as np
import dlib
import joblib
import mediapipe as mp
import time

# Initialize dlib's face detector and shape predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Function to detect facial landmarks from an image
def get_landmarks(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    rects = detector(gray, 1)
    
    if len(rects) > 0:
        # Assuming one face per image
        shape = predictor(gray, rects[0])
        landmarks = np.array([[p.x, p.y] for p in shape.parts()])
        return landmarks.flatten()  # Flatten the landmarks into a 1D array
    else:
        return None

# Load your pre-trained SVM models
svr_pitch = joblib.load('svr_pitch_model.pkl')
svr_yaw = joblib.load('svr_yaw_model.pkl')
svr_roll = joblib.load('svr_roll_model.pkl')

# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(max_num_faces=1, min_detection_confidence=0.5)

mp_drawing = mp.solutions.drawing_utils
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

# Start capturing video from the webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame.")
        break

    # Get landmarks using dlib and predict angles using SVM models
    landmarks = get_landmarks(frame)
    if landmarks is not None:
        pitch_pred = svr_pitch.predict([landmarks])[0]
        yaw_pred = svr_yaw.predict([landmarks])[0]
        roll_pred = svr_roll.predict([landmarks])[0]
    else:
        continue

    # Use MediaPipe to get face landmarks and calculate pose direction
    start = time.time()

    # Convert BGR to RGB
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image_rgb.flags.writeable = False
    results = face_mesh.process(image_rgb)
    image_rgb.flags.writeable = True
    frame = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)

    img_h, img_w, img_c = frame.shape

    if results.multi_face_landmarks:
        for face_idx, face_landmarks in enumerate(results.multi_face_landmarks):
            face_3d = []
            face_2d = []
            
            for idx, lm in enumerate(face_landmarks.landmark):
                if idx == 33 or idx == 263 or idx == 1 or idx == 61 or idx == 291 or idx == 199:
                    if idx == 1:
                        nose_2d = (lm.x * img_w, lm.y * img_h)
                        nose_3d = (lm.x * img_w, lm.y * img_h, lm.z * 3000)
                        
                    x, y = int(lm.x * img_w), int(lm.y * img_h)
                    
                    # 2D coordinates
                    face_2d.append([x, y])
                    
                    # 3D coordinates
                    face_3d.append([x, y, lm.z])
                    
            # Convert lists to numpy arrays
            face_2d = np.array(face_2d, dtype=np.float64)
            face_3d = np.array(face_3d, dtype=np.float64)
            
            # Camera matrix
            focal_length = 1 * img_w
            cam_mtx = np.array([[focal_length, 0, img_h / 2],
                                [0, focal_length, img_w / 2],
                                [0, 0, 1]])
            
            # Distortion parameters
            dist_mtx = np.zeros((4, 1), dtype=np.float64)
            
            # Solve PnP
            success, rot_vec, translation_vector = cv2.solvePnP(face_3d, face_2d, cam_mtx, dist_mtx)
            
            # Rotation matrix
            rmat, _ = cv2.Rodrigues(rot_vec)
            
            # Angles calculated from MediaPipe landmarks
            angles, mtxR, mtxQ, Qx, Qy, Qz = cv2.RQDecomp3x3(rmat)
            
            # Rotation degree
            x = pitch_pred * 180 / np.pi
            y = -yaw_pred * 180 / np.pi
            z = roll_pred * 180 / np.pi
            
            # Tilting position
            # if y < -10:
            #     text = f"Face {face_idx + 1}: Looking Left"
            # elif y > 10:
            #     text = f"Face {face_idx + 1}: Looking Right"
            # elif x < -10:
            #     text = f"Face {face_idx + 1}: Looking Down"
            # elif x > 10:
            #     text = f"Face {face_idx + 1}: Looking Up"
            # else:
            #     text = f"Face {face_idx + 1}: Forward"
                
            # Nose direction
            nose_3d_projection, _ = cv2.projectPoints(nose_3d, rot_vec, translation_vector, cam_mtx, dist_mtx)
            
            p1 = (int(nose_2d[0]), int(nose_2d[1]))
            
            # Fixed length for the blue line
            length = 150
            p2 = (int(nose_2d[0] + length * np.sin(y * np.pi / 180)), int(nose_2d[1] - length * np.sin(x * np.pi / 180))) 
            
            # Draw axis lines
            cv2.line(frame, p1, p2, (255, 0, 0), 3)  # Blue line
            cv2.line(frame, p1, (p1[0], p1[1] + 100), (0, 255, 0), 3)  # Green line
            cv2.line(frame, p1, (p1[0] + 100, p1[1]), (0, 0, 255), 3)  # Red line
            
            # Text on image with smaller font size
            font_scale = 0.4
            thickness = 1
            
            # cv2.putText(frame, text, (50, 50 + face_idx * 110), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (0, 0, 255), thickness)
            # cv2.putText(frame, f"X: {x:.2f} Y: {y:.2f} Z: {z:.2f}", (50, 80 + face_idx * 110), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (0, 0, 255), thickness)
            # cv2.putText(frame, f"SVM Pitch: {pitch_pred:.2f}, Yaw: {yaw_pred:.2f}, Roll: {roll_pred:.2f}", (50, 110 + face_idx * 110), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (0, 0, 255), thickness)
        
        end = time.time()
        totalTime = end - start

# Avoid division by zero
        if totalTime > 0:
            fps = 1 / totalTime
        else:
            fps = 0  # Or set to a default value if necessary

        print("FPS: ", fps)

        cv2.putText(frame, f"FPS: {int(fps)}", (50, 140 + len(results.multi_face_landmarks) * 110), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (0, 0, 255), thickness)

    cv2.imshow("Head Pose Estimation", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

c:\Users\0king\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


FPS:  62.352143664149374


c:\Users\0king\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\0king\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


FPS:  142.84803487500852
FPS:  142.91130873283586


c:\Users\0king\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\0king\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


...

FPS:  76.94276489580275


c:\Users\0king\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


FPS:  104.43463970917783


c:\Users\0king\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


FPS:  142.77995642701526
